사용 model : xgb, lgbm, cat, RF, inceptionV3  
  
table데이터만을 사용한 모델과 이미지만을 사용한 모델을 각각 만든 후 예측  
예측한 결과(확률)를 비율로 섞어 최종 예측을 만든 후 제출  
table 데이터 : 80% 이미지 데이터 : 20%

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import albumentations as A
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

GPU 사용

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11609658820907190276
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4850712576
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17960540280816153002
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

Data받기(수술연월일 = parse_date로 받기)

In [3]:
path = 'D:/Dataset/Competitons/cancer/'
train = pd.read_csv(path + 'train.csv', parse_dates=["수술연월일"]).copy()
test = pd.read_csv(path + 'test.csv', parse_dates=["수술연월일"]).copy()
clinical_info = pd.read_excel(path + 'clinical_info.xlsx').copy()

In [4]:
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0


비어있는 값 확인

In [5]:
train.isna().sum()

ID                     0
img_path               0
mask_path              0
나이                     0
수술연월일                  0
진단명                    0
암의 위치                  0
암의 개수                  0
암의 장경                 69
NG                    51
HG                    86
HG_score_1            92
HG_score_2            92
HG_score_3            89
DCIS_or_LCIS_여부        0
DCIS_or_LCIS_type    874
T_category             4
ER                     1
ER_Allred_score      297
PR                     1
PR_Allred_score      453
KI-67_LI_percent     235
HER2                  14
HER2_IHC              24
HER2_SISH            753
HER2_SISH_ratio      825
BRCA_mutation        945
N_category             0
dtype: int64

column 이름 확인

In [6]:
train.columns.unique()

Index(['ID', 'img_path', 'mask_path', '나이', '수술연월일', '진단명', '암의 위치', '암의 개수',
       '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
       'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER',
       'ER_Allred_score', 'PR', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2',
       'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation',
       'N_category'],
      dtype='object')

'수술연월일'을 year, month, day, quarter로 쪼개서 새 columns로 추가

In [7]:
def date_split(df):

    df['year'] = df['수술연월일'].dt.year
    df['month'] = df['수술연월일'].dt.month
    df['day'] = df['수술연월일'].dt.day
    df['quarter'] = df['수술연월일'].dt.quarter

    return df
    
train = date_split(train)
test = date_split(test)
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,year,month,day,quarter
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,0.0,1.0,NaN,NaN,NaN,0,2015,10,23,4
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,0.0,0.0,NaN,NaN,NaN,1,2015,10,28,4
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,0.0,1.0,NaN,NaN,0.0,0,2015,10,29,4
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,1.0,3.0,NaN,NaN,NaN,0,2016,3,8,1
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,1.0,2.0,1.0,5.44,NaN,0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,0.0,0.0,NaN,NaN,NaN,1,2006,12,22,4
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0,2006,11,17,4
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,0.0,0.0,NaN,NaN,NaN,1,2006,11,10,4
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0,2006,9,22,3


비어있는 값 채우기

In [8]:
train['암의 장경'] = train['암의 장경'].fillna(train['암의 장경'].mean())
train = train.fillna(0)

test['암의 장경'] = test['암의 장경'].fillna(train['암의 장경'].mean())
test = test.fillna(0)

In [9]:
train.isna().sum()

ID                   0
img_path             0
mask_path            0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
N_category           0
year                 0
month                0
day                  0
quarter              0
dtype: int64

In [10]:
test.isna().sum()

ID                   0
img_path             0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
year                 0
month                0
day                  0
quarter              0
dtype: int64

label값 복사

In [11]:
label = train['N_category'].values.copy()

In [12]:
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,year,month,day,quarter
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.000000,2.0,...,0.0,1.0,0.0,0.00,0.0,0,2015,10,23,4
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.000000,3.0,...,0.0,0.0,0.0,0.00,0.0,1,2015,10,28,4
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,18.375564,2.0,...,0.0,1.0,0.0,0.00,0.0,0,2015,10,29,4
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.000000,3.0,...,1.0,3.0,0.0,0.00,0.0,0,2016,3,8,1
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.000000,2.0,...,1.0,2.0,1.0,5.44,0.0,0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,1,2006,12,22,4
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,0,2006,11,17,4
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,1,2006,11,10,4
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,0,2006,9,22,3


ID, mask_path 제거 후 각각 train_table, test_table로 저장  
현재의 모델은 table데이터들만 사용하는 모델이지만, img_path는 나중에 따로 뺀 후 제거 예정

In [13]:
ignore_cols_train = ['ID', 'mask_path', '수술연월일', 'N_category']
ignore_cols_test = ['ID', 'img_path', '수술연월일']

train_table = train.drop(ignore_cols_train, axis = 1).copy()
test_table = test.drop(ignore_cols_test, axis = 1).copy()
train_table

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
0,./train_imgs/BC_01_0001.png,63,1,2,1,19.000000,2.0,1.0,2.0,2.0,...,12.0,0.0,1.0,0.0,0.00,0.0,2015,10,23,4
1,./train_imgs/BC_01_0002.png,51,1,1,1,22.000000,3.0,3.0,3.0,3.0,...,70.0,0.0,0.0,0.0,0.00,0.0,2015,10,28,4
2,./train_imgs/BC_01_0003.png,37,1,2,1,18.375564,2.0,0.0,0.0,0.0,...,7.0,0.0,1.0,0.0,0.00,0.0,2015,10,29,4
3,./train_imgs/BC_01_0004.png,54,1,2,1,0.000000,3.0,3.0,3.0,3.0,...,1.0,1.0,3.0,0.0,0.00,0.0,2016,3,8,1
4,./train_imgs/BC_01_0005.png,57,1,2,1,8.000000,2.0,2.0,3.0,2.0,...,8.0,1.0,2.0,1.0,5.44,0.0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,./train_imgs/BC_01_3464.png,65,1,2,1,25.000000,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,12,22,4
996,./train_imgs/BC_01_3482.png,48,1,1,1,7.000000,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,11,17,4
997,./train_imgs/BC_01_3485.png,64,1,2,1,15.000000,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,11,10,4
998,./train_imgs/BC_01_3502.png,50,1,1,1,7.000000,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,9,22,3


train_test_split으로 val데이터 생성(성능 check위해)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_table, label, random_state = 42)

In [15]:
X_train

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
82,./train_imgs/BC_01_0404.png,72,1,2,1,13.0,2.0,1.0,2.0,2.0,...,6.38,0.0,1.0,0.0,0.00,0.0,2019,12,26,4
991,./train_imgs/BC_01_3416.png,69,1,1,1,9.0,1.0,1.0,3.0,1.0,...,0.00,0.0,1.0,0.0,0.00,0.0,2009,5,6,2
789,./train_imgs/BC_01_2899.png,40,1,2,1,18.0,2.0,1.0,2.0,1.0,...,10.00,1.0,2.0,0.0,0.00,0.0,2008,5,27,2
894,./train_imgs/BC_01_3040.png,61,1,1,1,12.0,2.0,2.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,4,9,2
398,./train_imgs/BC_01_1939.png,40,1,1,1,1.5,1.0,1.0,3.0,1.0,...,5.00,0.0,1.0,0.0,0.00,0.0,2015,9,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,./train_imgs/BC_01_0530.png,54,1,2,1,15.0,3.0,3.0,3.0,3.0,...,24.15,0.0,2.0,0.0,1.44,0.0,2019,1,22,1
270,./train_imgs/BC_01_1252.png,56,1,1,1,22.0,3.0,2.0,2.0,3.0,...,11.71,0.0,2.0,0.0,1.90,0.0,2019,7,24,3
860,./train_imgs/BC_01_2997.png,62,1,2,1,12.0,3.0,2.0,3.0,3.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,7,20,3
435,./train_imgs/BC_01_2101.png,63,1,2,1,14.0,2.0,2.0,3.0,2.0,...,20.00,0.0,0.0,0.0,0.00,0.0,2015,3,13,1


train에 있는 img_path만 따로 저장(각각 X_train_img 와 X_val_img)  
현재 dataset은 코드저장되어있는 부분과 다른 곳에 위치하므로, path수정후 저장

In [16]:
X_train_img = X_train['img_path'].values.copy()
X_train_img = [path + j[2:] for j in X_train_img]
X_train_img = np.reshape(X_train_img, (len(X_train_img)))
X_train_img

X_val_img = X_val['img_path'].values.copy()
X_val_img = [path + j[2:] for j in X_val_img]
X_val_img = np.reshape(X_val_img, (len(X_val_img)))
X_val_img

array(['D:/Dataset/Competitons/cancer/train_imgs/BC_01_2385.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2822.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2825.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2726.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2017.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2746.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2680.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2364.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2994.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_0666.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2931.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_0382.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2693.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_3267.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_3134.png',
       'D:

img_path 따로 뺐으니, X_train, X_val에서 img_path 제거

In [17]:
ignore_img = ['img_path']

X_train = X_train.drop(ignore_img, axis = 1)
X_val = X_val.drop(ignore_img, axis = 1)

In [18]:
X_train

,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,HG_score_3,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
82,72,1,2,1,13.0,2.0,1.0,2.0,2.0,1.0,...,6.38,0.0,1.0,0.0,0.00,0.0,2019,12,26,4
991,69,1,1,1,9.0,1.0,1.0,3.0,1.0,1.0,...,0.00,0.0,1.0,0.0,0.00,0.0,2009,5,6,2
789,40,1,2,1,18.0,2.0,1.0,2.0,1.0,1.0,...,10.00,1.0,2.0,0.0,0.00,0.0,2008,5,27,2
894,61,1,1,1,12.0,2.0,2.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,4,9,2
398,40,1,1,1,1.5,1.0,1.0,3.0,1.0,1.0,...,5.00,0.0,1.0,0.0,0.00,0.0,2015,9,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,54,1,2,1,15.0,3.0,3.0,3.0,3.0,2.0,...,24.15,0.0,2.0,0.0,1.44,0.0,2019,1,22,1
270,56,1,1,1,22.0,3.0,2.0,2.0,3.0,1.0,...,11.71,0.0,2.0,0.0,1.90,0.0,2019,7,24,3
860,62,1,2,1,12.0,3.0,2.0,3.0,3.0,1.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,7,20,3
435,63,1,2,1,14.0,2.0,2.0,3.0,2.0,1.0,...,20.00,0.0,0.0,0.0,0.00,0.0,2015,3,13,1


In [19]:
len(X_train.columns.unique())

27

In [20]:
len(test.columns.unique())

30

In [73]:
len(test_table.columns.unique())

27

XGBClassifier  
xgboost가 없는 경우 : !pip install xgboost

In [21]:
from xgboost import XGBRFClassifier

xgb = XGBRFClassifier(random_state = 42)
xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)])

[0]	validation_0-logloss:0.40889


XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=0,
                num_parallel_tree=100, objective='binary:logistic',
                predictor='auto', random_state=42, reg_alpha=0,
                sampling_method='uniform', scale_pos_weight=1, ...)

test_table 예측 확인

In [22]:
xgb.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

test_table 예측 한 확률을 각각 출력후 저장  
예측 확률로 한 이유 ; 이후 각 예측 확률을 더해 최종 예측을 만들기 위해

In [23]:
xgb_pred = xgb.predict_proba(test_table)
xgb_pred

array([[0.13390356, 0.86609644],
       [0.6173823 , 0.38261768],
       [0.85107654, 0.14892346],
       [0.6649084 , 0.3350916 ],
       [0.4247328 , 0.5752672 ],
       [0.213601  , 0.786399  ],
       [0.2044928 , 0.7955072 ],
       [0.5892285 , 0.4107715 ],
       [0.40749246, 0.59250754],
       [0.483222  , 0.516778  ],
       [0.41854984, 0.58145016],
       [0.29913372, 0.7008663 ],
       [0.74090445, 0.25909555],
       [0.4924677 , 0.5075323 ],
       [0.71236587, 0.2876341 ],
       [0.15928519, 0.8407148 ],
       [0.25800234, 0.74199766],
       [0.7478635 , 0.25213653],
       [0.19576341, 0.8042366 ],
       [0.1916914 , 0.8083086 ],
       [0.2590503 , 0.7409497 ],
       [0.13977587, 0.8602241 ],
       [0.8799281 , 0.12007187],
       [0.14480323, 0.8551968 ],
       [0.8636033 , 0.1363967 ],
       [0.13620287, 0.8637971 ],
       [0.12824428, 0.8717557 ],
       [0.81033087, 0.18966915],
       [0.1310913 , 0.8689087 ],
       [0.7646071 , 0.2353929 ],
       [0.

CatBoostClassifier  
catboost가 없는 경우 : !pip install catboost

In [24]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(random_state = 42, verbose = 0)
cat.fit(X_train, y_train, eval_set = [(X_val, y_val)])

test_table cat으로 예측 확인

In [25]:
cat.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

예측 확률 저장

In [26]:
cat_pred = cat.predict_proba(test_table)
cat_pred

array([[0.1013835 , 0.8986165 ],
       [0.56269454, 0.43730546],
       [0.98826786, 0.01173214],
       [0.80606294, 0.19393706],
       [0.16994983, 0.83005017],
       [0.07053495, 0.92946505],
       [0.14451621, 0.85548379],
       [0.65530957, 0.34469043],
       [0.4106533 , 0.5893467 ],
       [0.19368345, 0.80631655],
       [0.44327537, 0.55672463],
       [0.30932796, 0.69067204],
       [0.55385243, 0.44614757],
       [0.31132559, 0.68867441],
       [0.82342115, 0.17657885],
       [0.03287513, 0.96712487],
       [0.06285034, 0.93714966],
       [0.92920599, 0.07079401],
       [0.07501022, 0.92498978],
       [0.23025559, 0.76974441],
       [0.28907812, 0.71092188],
       [0.07989782, 0.92010218],
       [0.98778209, 0.01221791],
       [0.02846503, 0.97153497],
       [0.9472887 , 0.0527113 ],
       [0.03430458, 0.96569542],
       [0.09326371, 0.90673629],
       [0.97191637, 0.02808363],
       [0.01558507, 0.98441493],
       [0.65034867, 0.34965133],
       [0.

ExtraTreesClassifier

In [27]:
from sklearn.ensemble import ExtraTreesClassifier

ex_clf = ExtraTreesClassifier(random_state = 42)
ex_clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=42)

In [28]:
ex_clf.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [29]:
ex_pred = ex_clf.predict_proba(test_table)
ex_pred

array([[0.36, 0.64],
       [0.36, 0.64],
       [0.98, 0.02],
       [0.69, 0.31],
       [0.17, 0.83],
       [0.11, 0.89],
       [0.25, 0.75],
       [0.55, 0.45],
       [0.43, 0.57],
       [0.18, 0.82],
       [0.43, 0.57],
       [0.51, 0.49],
       [0.27, 0.73],
       [0.48, 0.52],
       [0.73, 0.27],
       [0.28, 0.72],
       [0.14, 0.86],
       [0.83, 0.17],
       [0.27, 0.73],
       [0.24, 0.76],
       [0.26, 0.74],
       [0.2 , 0.8 ],
       [1.  , 0.  ],
       [0.13, 0.87],
       [1.  , 0.  ],
       [0.08, 0.92],
       [0.18, 0.82],
       [0.97, 0.03],
       [0.03, 0.97],
       [0.23, 0.77],
       [0.04, 0.96],
       [0.87, 0.13],
       [0.08, 0.92],
       [0.07, 0.93],
       [0.08, 0.92],
       [0.03, 0.97],
       [1.  , 0.  ],
       [0.34, 0.66],
       [0.02, 0.98],
       [0.26, 0.74],
       [0.1 , 0.9 ],
       [0.08, 0.92],
       [0.06, 0.94],
       [0.06, 0.94],
       [0.18, 0.82],
       [0.19, 0.81],
       [0.09, 0.91],
       [0.33,

RandomForestClassifier

In [30]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [31]:
rf.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [32]:
rf_pred = rf.predict_proba(test_table)
rf_pred

array([[0.17, 0.83],
       [0.49, 0.51],
       [0.97, 0.03],
       [0.62, 0.38],
       [0.22, 0.78],
       [0.15, 0.85],
       [0.29, 0.71],
       [0.52, 0.48],
       [0.26, 0.74],
       [0.37, 0.63],
       [0.36, 0.64],
       [0.44, 0.56],
       [0.48, 0.52],
       [0.28, 0.72],
       [0.76, 0.24],
       [0.12, 0.88],
       [0.16, 0.84],
       [0.81, 0.19],
       [0.15, 0.85],
       [0.25, 0.75],
       [0.42, 0.58],
       [0.12, 0.88],
       [1.  , 0.  ],
       [0.09, 0.91],
       [0.95, 0.05],
       [0.08, 0.92],
       [0.14, 0.86],
       [0.97, 0.03],
       [0.04, 0.96],
       [0.43, 0.57],
       [0.07, 0.93],
       [0.87, 0.13],
       [0.09, 0.91],
       [0.08, 0.92],
       [0.15, 0.85],
       [0.05, 0.95],
       [0.95, 0.05],
       [0.22, 0.78],
       [0.05, 0.95],
       [0.18, 0.82],
       [0.07, 0.93],
       [0.1 , 0.9 ],
       [0.08, 0.92],
       [0.01, 0.99],
       [0.23, 0.77],
       [0.24, 0.76],
       [0.08, 0.92],
       [0.2 ,

LGBMClassifier  
lightgbm이 없는 경우 : !pip install lightgbm

In [33]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state = 42)
lgbm.fit(X_train, y_train)

LGBMClassifier(random_state=42)

In [34]:
lgbm.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [35]:
lgbm_pred = lgbm.predict_proba(test_table)
lgbm_pred

array([[6.46172150e-02, 9.35382785e-01],
       [7.60264882e-01, 2.39735118e-01],
       [9.99497074e-01, 5.02925580e-04],
       [9.19409267e-01, 8.05907331e-02],
       [7.20652751e-02, 9.27934725e-01],
       [1.09662099e-01, 8.90337901e-01],
       [1.92210382e-02, 9.80778962e-01],
       [6.21541757e-01, 3.78458243e-01],
       [4.98724357e-01, 5.01275643e-01],
       [9.87878364e-02, 9.01212164e-01],
       [4.90633430e-01, 5.09366570e-01],
       [5.38028363e-01, 4.61971637e-01],
       [8.66514025e-01, 1.33485975e-01],
       [5.06552448e-01, 4.93447552e-01],
       [5.75803781e-01, 4.24196219e-01],
       [2.89605773e-02, 9.71039423e-01],
       [2.31353663e-02, 9.76864634e-01],
       [9.79864057e-01, 2.01359428e-02],
       [5.35117947e-03, 9.94648821e-01],
       [2.06970313e-01, 7.93029687e-01],
       [3.19711066e-01, 6.80288934e-01],
       [8.38101198e-02, 9.16189880e-01],
       [9.99489409e-01, 5.10590566e-04],
       [3.91122707e-03, 9.96088773e-01],
       [9.866526

f1_score확인  
성능 확인 방법 : model에 넣지 않은 val을 넣고 성능 확인

In [36]:
from sklearn.metrics import f1_score
def scoring(y, pred):
    score = f1_score(y_true = y, y_pred = pred)
    print(score)

 score찍어서 확인

In [37]:
model = [xgb, cat, rf, ex_clf, lgbm]
for i in model:
    print(str(i))
    print(i.score(X_val, y_val))

XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=0,
                num_parallel_tree=100, objective='binary:logistic',
                predictor='auto', random_state=42, reg_alpha=0,
                sampling_method='uniform', scale_pos_weight=1, ...)
0.84
0.828
RandomForestClassifier(random_state=42)
0.82
ExtraTreesClassifier(random_state=42)
0.812
LGBMClassifier(random_state=42)
0.784


f1_score 점수 확인

In [38]:
for i in model:
    print(str(i))
    print(scoring(y_val, i.predict(X_val)))

XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=0,
                num_parallel_tree=100, objective='binary:logistic',
                predictor='auto', random_state=42, reg_alpha=0,
                sampling_method='uniform', scale_pos_weight=1, ...)
0.83739837398374
None
0.8326848249027238
None
RandomForestClassifier(random_state=42)
0.8221343873517787
None
ExtraTreesClassifier(random_state=42)
0.8142292490118578
None
LGBMClassifier(random_state=42)
0.7969924812030075
None


Extra는 RF와 비슷하므로 둘 중 낮은 값인 Extra는 사용 X

Image모델(InceptionV3 사용)

In [39]:
# Image

test의 img_path 역시 dataset이 위치한 곳으로 수정 후 따로 저장

In [40]:
test_img = test['img_path'].values.copy()
test_img = [path + j[2:] for j in test_img]
test_img = np.reshape(test_img, (len(test_img)))
test_img

array(['D:/Dataset/Competitons/cancer/test_imgs/BC_01_0011.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0220.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0233.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0258.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0260.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0266.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0269.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0272.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0275.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0278.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0279.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0286.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0296.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0303.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0310.png',
       'D:/Dataset/Compet

In [41]:
X_train_img[0]

'D:/Dataset/Competitons/cancer/train_imgs/BC_01_0404.png'

In [42]:
len(X_train_img)

750

In [43]:
type(X_train_img)

numpy.ndarray

In [44]:
type(y_train)

numpy.ndarray

img_size 지정 img_resize

In [45]:
img_size = 512
img_channel = 3
batch = 8

resize_crop = A.Compose([
    A.Resize(img_size, img_size, interpolation = cv2.INTER_AREA)
])

CustomSequence제작  
img dataset을 사용한 이유 : 데이터 양이 너무 커서 컴퓨터가 버티질 못해 사용....ㅠ  
dataset을 사용하니 메모리가 터지지는 않으나, 너무 느려짐.....ㅠㅜㅠㅜㅠ

In [46]:
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
from tensorflow.keras.applications.xception import preprocess_input
import math

class CustomSequence(Sequence):
    def __init__(self, filenames, target, batch_size, aug=None, shuffle=False):
        # image의 절대경로들
        self.filenames = filenames
        self.target = target
        self.batch_size = batch_size
        # albumentation 객체
        self.aug = aug
        self.shuffle = shuffle
        
        # 훈련 데이터의 경우
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        # 총 step의 갯수
        return math.ceil(len(self.filenames) / self.batch_size)

    def __getitem__(self, index):
        # 현재 인덱스를 기준으로 batch_size만큼 데이터를 가져옴
        meta_data = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # 훈련, 검증 데이터세트인 경우
        if self.target is not None:
            label_batch = self.target[index*self.batch_size:(index+1)*self.batch_size]

        # 불러온 meta_data를 np.array로 저장할 빈 공간을 생성
        image_batch = np.zeros((meta_data.shape[0], img_size, img_size, img_channel))
        for i in range(meta_data.shape[0]):
            # cv2는 이미지를 BGR로 불러오기 때문이 RGB로 바꾸어줌
            image = cv2.cvtColor(cv2.imread(meta_data[i]), cv2.COLOR_BGR2RGB)
            # augmentation이 있으면 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']

            # 이미지 값을 self.pre_func 함수로 스케일링
            image = image * 1./255

            # 빈 이미지 배치에 최종 이미지를 등록
            image_batch[i] = image
            
        if self.target is not None:
            return image_batch, label_batch
        else:
            return image_batch
        
    def on_epoch_end(self):
        # 파일과 라벨을 같이 섞어 주어야한다.
        if self.shuffle is True:
            self.image_filenames, self.target = shuffle(self.filenames, self.target)
        else:
            pass

In [47]:
train_dataset = CustomSequence(X_train_img, y_train, batch, resize_crop)
val_dataset = CustomSequence(X_val_img, y_val, batch, resize_crop)

In [48]:
len(train_dataset)

94

dataset이 잘 되었는지 확인

In [49]:
def show_first_data(dataset, image_verbose=False):
    first_data = next(iter(dataset))
    images = first_data[0]
    labels = first_data[1]
    print(images.shape, labels.shape)
    print(images[0])
    if image_verbose:
        fig, axs = plt.subplots(figsize=(20, 8), nrows=1, ncols=4)
        for i in range(4):
            axs[i].imshow(np.array(images[i], dtype='int32'))

In [50]:
show_first_data(train_dataset)

(8, 512, 512, 3) (8,)
[[[0.95686275 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 [[0.95686275 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.96078431]
  [0.95686275 0.96078431 0.95686275]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 [[0.95686275 0.95686275 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.96078431]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 ...

 [[0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  ...
  [0.95294118 0.95686275 0.95294118]
  [0.95294118 0.95294118 0.95294118]
  [0.95294118 0.95294118 0.95294118]]

 [[0.95686275 0.95686275 0.95686275]
  [0.95686275 0

CNN 모델 생성(inceptionV3)

In [51]:
def make_model():
    base_model = keras.applications.InceptionV3(include_top = False, weights = 'imagenet', input_shape = (img_size, img_size, 3), pooling = None)
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(512, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(256, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(128, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(64, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    predictions = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=predictions)    
    return model

In [52]:
img_model = make_model()

In [53]:
img_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 255, 255, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

                                                                                                  
 conv2d_10 (Conv2D)             (None, 61, 61, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 61, 61, 32)   6144        ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 61, 61, 64)  192         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 61, 61, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

                                                                  'activation_14[0][0]',          
                                                                  'activation_17[0][0]',          
                                                                  'activation_18[0][0]']          
                                                                                                  
 conv2d_22 (Conv2D)             (None, 61, 61, 64)   18432       ['mixed1[0][0]']                 
                                                                                                  
 batch_normalization_22 (BatchN  (None, 61, 61, 64)  192         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 61, 61, 64)   0           ['batch_normalization_22[0][0]'] 
          

 ormalization)                                                                                    
                                                                                                  
 activation_26 (Activation)     (None, 30, 30, 384)  0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 30, 30, 96)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 30, 30, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 30, 30, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
          

 activation_39 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 mixed4 (Concatenate)           (None, 30, 30, 768)  0           ['activation_30[0][0]',          
                                                                  'activation_33[0][0]',          
                                                                  'activation_38[0][0]',          
                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 30, 30, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 30, 30, 160)  480        ['conv2d_44[0][0]']              
 ormalizat

                                                                  'activation_49[0][0]']          
                                                                                                  
 conv2d_54 (Conv2D)             (None, 30, 30, 160)  122880      ['mixed5[0][0]']                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 30, 30, 160)  480        ['conv2d_54[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 30, 30, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 30, 30, 160)  179200      ['activation_54[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_64 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_64[0][0]'] 
                                                                                                  
 conv2d_65 (Conv2D)             (None, 30, 30, 192)  258048      ['activation_64[0][0]']          
                                                                                                  
 batch_normalization_65 (BatchN  (None, 30, 30, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_65[0][0]'] 
          

                                                                                                  
 batch_normalization_73 (BatchN  (None, 30, 30, 192)  576        ['conv2d_73[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_73 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_73[0][0]'] 
                                                                                                  
 conv2d_70 (Conv2D)             (None, 30, 30, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 30, 30, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_nor

 batch_normalization_83 (BatchN  (None, 14, 14, 384)  1152       ['conv2d_83[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_84 (Conv2D)             (None, 14, 14, 192)  245760      ['average_pooling2d_7[0][0]']    
                                                                                                  
 batch_normalization_76 (BatchN  (None, 14, 14, 320)  960        ['conv2d_76[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_78[0][0]'] 
                                                                                                  
 activatio

 ormalization)                                                                                    
                                                                                                  
 activation_87 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_87[0][0]'] 
                                                                                                  
 activation_88 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_88[0][0]'] 
                                                                                                  
 activation_91 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_91[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 batch_nor

callback함수(ReduceLROnPlateau, EarlyStopping, Checkpoint)  
  
val로 성능 확인하면서 3번 이상 성능 감소 안하면 learning late감소, 10번 val 성능이 감소 안하면 정지,  
가장 좋은 모델로 callback)  

In [54]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode = 'min')
ES = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint_cb = keras.callbacks.ModelCheckpoint('img_model.h5', save_best_only=True)

In [55]:
img_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

img_model.fit(train_dataset, batch_size = batch,
              validation_data = val_dataset,
              epochs=100, callbacks=[reduce_lr, ES, checkpoint_cb],
              verbose = 1)

Epoch 1/100
94/94 [==============================] - 274s 3s/step - loss: 0.8700 - acc: 0.5560 - val_loss: 219252.7188 - val_acc: 0.5000 - lr: 0.0010
Epoch 2/100
94/94 [==============================] - 261s 3s/step - loss: 0.7211 - acc: 0.5240 - val_loss: 0.7009 - val_acc: 0.5000 - lr: 0.0010
Epoch 3/100
94/94 [==============================] - 262s 3s/step - loss: 0.7064 - acc: 0.4960 - val_loss: 0.6932 - val_acc: 0.5000 - lr: 0.0010
Epoch 4/100
94/94 [==============================] - 260s 3s/step - loss: 0.7218 - acc: 0.5013 - val_loss: 9.9326 - val_acc: 0.4960 - lr: 0.0010
Epoch 5/100
94/94 [==============================] - 259s 3s/step - loss: 0.7140 - acc: 0.4840 - val_loss: 0.7737 - val_acc: 0.5000 - lr: 0.0010
Epoch 6/100
94/94 [==============================] - 267s 3s/step - loss: 0.7156 - acc: 0.4947 - val_loss: 0.6919 - val_acc: 0.5000 - lr: 0.0010
Epoch 7/100
94/94 [==============================] - 271s 3s/step - loss: 0.7042 - acc: 0.5160 - val_loss: 199.5185 - val_acc

epoch가 더 늘어날 수도, 줄어들 수도 있음(제출 epoch는 16에서 정지 -> epoch = 6)

In [56]:
test_img.shape

(250,)

test_img도 dataset으로 제작

In [57]:
test_dataset = CustomSequence(test_img, None, batch, resize_crop)

In [58]:
test_dataset

CNN에서 predict를 하면, 0에서 1 사이의 값이 나온다.(확률)  
이는 1로 될 확률과 같으므로, 0일 때의 확률은 1 - predict이고, 1일 때의 확률은 predict로 하는 array를 생성한다.

In [59]:
def img_predict_proba(model, test_dataset):
    img_pred = model.predict(test_dataset)
    pred_test = np.zeros((len(img_pred), 2))

    for i in range(0, len(img_pred)):
        pred_test[i][0] = (1 - img_pred[i])
        pred_test[i][1] = img_pred[i]

    return pred_test

In [60]:
img_pred = img_predict_proba(img_model, test_dataset)

32/32 [==============================] - 71s 2s/step


val_dataset으로 f1_score 확인

In [70]:
scoring(y_val, np.argmax(img_predict_proba(img_model, val_dataset), axis = 1))

32/32 [==============================] - 67s 2s/step
0.6491228070175439


xgb, rf, cat, lgbm과 img 확률들을 더해 최종 예측 생성(각각 0.2씩)

In [62]:
pred = (img_pred + xgb_pred + rf_pred + cat_pred + lgbm_pred) / 5
pred

array([[0.19063903, 0.80936097],
       [0.58272717, 0.41727283],
       [0.85887134, 0.14112866],
       [0.69889047, 0.30110953],
       [0.29028152, 0.70971847],
       [0.20541953, 0.79458047],
       [0.22830198, 0.77169802],
       [0.57388177, 0.42611823],
       [0.42375699, 0.57624301],
       [0.32595829, 0.67404171],
       [0.43383614, 0.56616386],
       [0.41395686, 0.58604314],
       [0.62490979, 0.37509021],
       [0.41472971, 0.58527029],
       [0.67097831, 0.32902169],
       [0.16492739, 0.83507261],
       [0.19745617, 0.80254383],
       [0.79005657, 0.20994343],
       [0.18188395, 0.81811605],
       [0.27346852, 0.72653148],
       [0.35422842, 0.64577158],
       [0.18150369, 0.81849631],
       [0.87009746, 0.12990254],
       [0.16555071, 0.83444929],
       [0.86220293, 0.13779708],
       [0.14746462, 0.85253538],
       [0.17770622, 0.82229378],
       [0.84214738, 0.15785262],
       [0.13534907, 0.86465093],
       [0.65787825, 0.34212174],
       [0.

0일때의 확률과 1일 때의 확률이 1인지 확인

In [63]:
for i in range(0, len(pred)):
    print(pred[i][0] + pred[i][1])

1.0
0.9999999940395354
1.0
0.9999999999999999
0.9999999940395354
1.0
1.0
1.0
0.9999999940395354
1.0
1.0
1.0
1.0
1.0
0.9999999940395357
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999970197677
1.0
1.0000000059604646
0.9999999999999999
0.9999999999999999
1.0000000029802323
1.0
0.9999999940395354
1.0
1.0000000044703483
1.0
0.9999999999999999
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0000000044703485
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.000000001490116
0.9999999955296516
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0000000000000002
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0000000059604646
1.0
1.0
1.0
0.9999999999999999
1.0000000059604646
1.000000001490116
0.9999999999999999
0.9999999970197677
0.9999999970197678
0.9999999970197679
1.0
0.9999999940395357
0.9999999999999999
1.0000000044703483
0.999999998509884
0.9999999999999999
1.0
0.9999999940395355
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.00000000

이정도 오차는 괜춘!

In [64]:
len(pred)

250

np.argmax를 이용해 확인

In [65]:
np.argmax(img_pred, axis = 1)

array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [66]:
np.argmax(pred, axis = 1)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [67]:
np.argmax(rf_pred, axis = 1)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

submission csv생성

In [69]:
output = pd.DataFrame({'ID' : test['ID'], 'N_category':np.argmax(pred, axis = 1)})
output.to_csv('final_school.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
